In [49]:
from WindPy import w
w.start()
w.isconnected()
import rqdatac
rqdatac.init()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2021 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


/Users/domi/Python/.conda/lib/python3.11/site-packages/rqdatac/client.py:218: UserWarning: rqdatac is already inited. Settings will be changed.
  warnings.warn("rqdatac is already inited. Settings will be changed.", stacklevel=0)
/Users/domi/Python/.conda/lib/python3.11/site-packages/rqdatac/client.py:257: UserWarning: Your account will be expired after  29 days. Please call us at 0755-22676337 to upgrade or purchase or renew your contract.
  warnings.warn("Your account will be expired after  {} days. "


In [50]:
'''
【回测设定】
回测标的：上证 50ETF
时间区间：2015-02-09 至 2024-12-31
保证金占用：ETF 期权的备兑策略可用持仓 ETF 冲抵保证金，无需缴纳多余保证金，
移仓判断：当所持有期权合约到期期限不足 8 个交易日时，平仓并在次月合约上开仓
开仓方式：在开盘时以开盘价开仓
交易成本：对于期权合约，单边 6.6 元（佣金 5 元+交易所手续费 1.3 元+交易所结算
费 0.3 元），开仓平仓各收取一次；对于 ETF，买入和卖出时各收取 0.5%的申赎费。暂
不考虑滑点。
分红处理：考虑到 ETF 分红时备兑持仓需补券，默认 ETF 分红部分再投资买入 ETF。
【策略一】期初持有 10000 份上证 50ETF，并每月以平值/虚一档/虚二档卖出同等份额
的 ETF 近月看涨期权。
'''

'\n【回测设定】\n回测标的：上证 50ETF\n时间区间：2015-02-09 至 2024-12-31\n保证金占用：ETF 期权的备兑策略可用持仓 ETF 冲抵保证金，无需缴纳多余保证金，\n移仓判断：当所持有期权合约到期期限不足 8 个交易日时，平仓并在次月合约上开仓\n开仓方式：在开盘时以开盘价开仓\n交易成本：对于期权合约，单边 6.6 元（佣金 5 元+交易所手续费 1.3 元+交易所结算\n费 0.3 元），开仓平仓各收取一次；对于 ETF，买入和卖出时各收取 0.5%的申赎费。暂\n不考虑滑点。\n分红处理：考虑到 ETF 分红时备兑持仓需补券，默认 ETF 分红部分再投资买入 ETF。\n【策略一】期初持有 10000 份上证 50ETF，并每月以平值/虚一档/虚二档卖出同等份额\n的 ETF 近月看涨期权。\n'

In [51]:
import datetime
import calendar

def fourth_wednesday(yymm):
    # 解析输入的 yymm 格式
    year = int(yymm[:2])
    month = int(yymm[2:])
    # 如果年份是两位数，转换为四位数
    if year < 100:
        year += 2000  # 假设年份在2000年到2099年之间
    # 获取该月份的第一天是星期几（0=星期一，6=星期日）
    first_day_weekday, _ = calendar.monthrange(year, month)
    # 计算第一个星期三
    first_wednesday = 2 - first_day_weekday+1  # 2代表星期三
    if first_wednesday < 0:
        first_wednesday += 7
    # 计算第四个星期三
    fourth_wednesday = first_wednesday + 21  # 21天后是第四个星期三
    # 创建日期对象
    result_date = datetime.date(year, month, fourth_wednesday)
    return result_date

In [52]:
# 名义本金 100万
principal = 1000000
underlying_etf = "510050.XSHG"
# 从2016-01-01开始，持有10000份上证50etf，持有至2024-12-31
df_close_etf=rqdatac.fund.get_nav("510050",expect_df=True,start_date=20150209,end_date=20241231)
df_close_etf = df_close_etf.dropna(axis=1,how='all')
# 期初持有1万份上证50etf，以单位净值买入，而净值变化需要根据复权净值计算
# df_close_etf新增一列，列名为net_value_test
# 只保留df_close_etf的第二重日期索引
df_etf = df_close_etf.droplevel(0)
df_etf['net_value_from1'] = df_etf["adjusted_net_value"]/df_etf["adjusted_net_value"][0]
df_etf['net_value_hold'] = df_etf['net_value_from1']*df_etf["unit_net_value"][0]
df_etf

,unit_net_value,acc_net_value,change_rate,adjusted_net_value,net_value_from1,net_value_hold
datetime,,,,,,
2015-02-09,2.3330,3.1060,0.017444,3.331369,1.000000,2.333000
2015-02-10,2.3690,3.1490,0.015431,3.382775,1.015431,2.369000
2015-02-11,2.3810,3.1630,0.005065,3.399910,1.020574,2.381000
2015-02-12,2.3830,3.1660,0.000840,3.402766,1.021432,2.383000
2015-02-13,2.3930,3.1770,0.004196,3.417045,1.025718,2.393000
...,...,...,...,...,...,...
2024-12-25,2.7641,4.1211,0.002903,4.600187,1.380870,3.221569
2024-12-26,2.7565,4.1121,-0.002749,4.587539,1.377073,3.212712
2024-12-27,2.7556,4.1110,-0.000327,4.586041,1.376624,3.211663


In [53]:
from tqdm import tqdm
import pandas as pd
import dateutil
for date,row in tqdm(df_etf[:100].iterrows()):
    # 计算每个交易日10000份50etf的资产净值
    asset_etf_daily = 10000* row["net_value_hold"]
    # 获取当日50etf近月合约的合约代码
    # 获取某日期一个月后的日期
    option_code_list = rqdatac.options.get_contracts(underlying=underlying_etf,option_type="C",\
    maturity=(date+dateutil.relativedelta.relativedelta(months=1)).strftime("%y%m"),trading_date=df_etf.index[0])
    option_info = rqdatac.options.get_contract_property(order_book_ids=option_code_list,start_date=df_etf.index[0],end_date=df_etf.index[0])
    option_info2 = rqdatac.instruments(option_code_list,market="cn")
    # option_info_atm = option_info[option_info["strike_price"]<=row["unit_net_value"]].iloc[-1]
    break

0it [00:04, ?it/s]


In [60]:
mat_date_list = []
for _,row_info in option_info.iterrows():
    mat_date_list.append(rqdatac.instruments(row_info.name,market="cn")[0].maturity_date)
option_info["maturity_date"] = mat_date_list
option_info["label"] = ""
atm_row = option_info.loc[option_info[option_info["strike_price"]<row["unit_net_value"]]["strike_price"].idxmax()]
atm_row

product_name           510050C1503M02300
symbol                      50ETF购3月2300
contract_multiplier                10000
strike_price                         2.3
maturity_date                 2015-03-25
label                                   
Name: (10000003, 2015-02-09 00:00:00), dtype: object

In [48]:
option_info

,,product_name,symbol,contract_multiplier,strike_price,maturity_date
order_book_id,trading_date,,,,,
10000001,2015-02-09,510050C1503M02200,50ETF购3月2200,10000,2.20,2015-03-25
10000002,2015-02-09,510050C1503M02250,50ETF购3月2250,10000,2.25,2015-03-25
10000003,2015-02-09,510050C1503M02300,50ETF购3月2300,10000,2.30,2015-03-25
10000004,2015-02-09,510050C1503M02350,50ETF购3月2350,10000,2.35,2015-03-25
10000005,2015-02-09,510050C1503M02400,50ETF购3月2400,10000,2.40,2015-03-25
